In [ ]:
# Imports
import glob
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.dates as mdates
import seaborn as sns
import statsmodels.api as sm
import shutil
import csv
import yaml
from scipy import stats
import math
import sys
import datetime as dt


In [ ]:
toptoplevel = os.path.abspath('..')
toplevel = os.path.dirname(toptoplevel)
if not toplevel in sys.path:
    sys.path.append(toplevel)

from df_loader import main
from df_loader import METRICS as COL_NAMES

In [ ]:
def prepareFolder(path):
    if os.path.exists(path+'/') == True:
        shutil.rmtree(path)
    if os.path.exists(path+'/') == False:
        os.makedirs(path+'/')


In [ ]:
dfs, names = main()
# sanity check
for (df, name) in zip(dfs, names):
    print(df['usr'].iloc[0], name)

prepareFolder('plots/timeseries')


In [ ]:
# Overhead No agent, no malware
plt.style.use('classic')

plt.rc('font', size=10)  # controls default text size
plt.rc('axes', titlesize=26)  # fontsize of the title
# plt.rc('axes', labelsize=26) #fontsize of the x and y labels
plt.rc('xtick', labelsize=10)  # fontsize of the x tick labels
plt.rc('ytick', labelsize=10)  # fontsize of the y tick labels
plt.rc('legend', fontsize=14)  # fontsize of the legend

df = dfs[0]
name = names[0]


start = df.index[0]
# print(df.index[0])
if not os.path.exists('plots/timeseries/'+name):
    os.mkdir('plots/timeseries/'+name)
else:
    shutil.rmtree('plots/timeseries/'+name)
    os.mkdir('plots/timeseries/'+name)
index = 0
if os.path.exists('avg-noStraSelA-noMalware.csv'):
    os.remove('avg-noStraSelA-noMalware.csv')
# MTD trigger point
for column in df.columns:
    # print(column)

    fig = plt.figure(figsize=(6.4, 4.8), dpi=600)
    df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # remove outliers

    # calculate average
    df[column+'-avg-healthy'] = df[column].mean()

    with open("avg-noStraSelA-noMalware.csv", "a", newline="") as avgs:
        csv_writer = csv.writer(avgs)
        csv_writer.writerows([[column, df[column].mean()]])

    df[column+'-rolling-avg'] = df[column].rolling(10).mean()
    df[column+'-rolling-non-overlapping-avg'] = df[column].rolling(10).mean()[
        10-1::10]
    df[column+'-rolling-non-overlapping-avg'].bfill(inplace=True)

    ax1 = plt.subplot(311)
    ax1 = df[column].plot(color='C0', alpha=1, linewidth=1,
                          label=COL_NAMES[index].split(':')[0])

    plt.xlabel('')  # override

    upperBound = df[column].max()
    lowerBound = df[column].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax1 = plt.ylim(bottom=-1, top=1)
    else:
        ax1 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.title('Metric {}'.format(column))

    plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right',
               borderaxespad=0, fancybox=True, framealpha=0.5)

    ax2 = plt.subplot(312)
    ax2 = df[column+'-avg-healthy'].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-rolling-non-overlapping-avg'].plot(
        color='C6', alpha=1, linewidth=1, label='rolling average')

    plt.xlabel('')  # override

    upperBound = df[column+'-rolling-non-overlapping-avg'].max()
    lowerBound = df[column+'-rolling-non-overlapping-avg'].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax2 = plt.ylim(bottom=-1, top=1)
    else:
        ax2 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)

    ax3 = plt.subplot(313)
    ax3 = plt.axvline(df.index[120], color='C2',
                      linewidth=0)  # triggering malware
    ax3 = df[column+'-avg-healthy'].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='average')

    ax3.xaxis.set_major_locator(mdates.MinuteLocator())
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    # adjust y axis
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax3 = plt.ylim(bottom=-1, top=1)
    else:
        ax3 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=True)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)
    plt.xlabel('')  # override
    #plt.legend(loc='upper left')

    fig.subplots_adjust(hspace=0.2, left=0.14, right=0.90)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=True)

    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    plt.legend(lines[1:], labels[1:], bbox_to_anchor=(
        0.0, -0.5), loc='upper left', borderaxespad=0, fancybox=True, framealpha=0.5)

    plt.savefig('plots/timeseries/{}/{}-{}-timeseries.pdf'.format(name, name[3:], column,),
                facecolor='white', transparent=False, bbox_inches="tight")
    index += 1

    plt.cla()
plt.close('all')


In [ ]:
# Overhead Agent, no malware
plt.style.use('classic')

plt.rc('font', size=10)  # controls default text size
plt.rc('axes', titlesize=26)  # fontsize of the title
# plt.rc('axes', labelsize=26) #fontsize of the x and y labels
plt.rc('xtick', labelsize=10)  # fontsize of the x tick labels
plt.rc('ytick', labelsize=10)  # fontsize of the y tick labels
plt.rc('legend', fontsize=14)  # fontsize of the legend

df = dfs[1]
name = names[1]


start = df.index[1]
# print(df.index[0])
if not os.path.exists('plots/timeseries/'+name):
    os.mkdir('plots/timeseries/'+name)
else:
    shutil.rmtree('plots/timeseries/'+name)
    os.mkdir('plots/timeseries/'+name)
index = 0


if os.path.exists('avg-StraSelA-noMalware.csv'):
    os.remove('avg-StraSelA-noMalware.csv')

# MTD trigger point
for column in df.columns:
    # print(column)

    fig = plt.figure(figsize=(6.4, 4.8), dpi=600)
    df[(np.abs(stats.zscore(df)) < 3).all(axis=1)]  # remove outliers

    # calculate average
    df[column+'-avg-healthy'] = df[column].mean()

    with open("avg-StraSelA-noMalware.csv", "a", newline="") as avgs:
        csv_writer = csv.writer(avgs)
        csv_writer.writerows([[column, df[column].mean()]])

    df[column+'-rolling-avg'] = df[column].rolling(10).mean()
    df[column+'-rolling-non-overlapping-avg'] = df[column].rolling(10).mean()[
        10-1::10]
    df[column+'-rolling-non-overlapping-avg'].bfill(inplace=True)

    ax1 = plt.subplot(311)
    ax1 = df[column].plot(color='C0', alpha=1, linewidth=1,
                          label=COL_NAMES[index].split(':')[0])

    plt.xlabel('')  # override

    upperBound = df[column].max()
    lowerBound = df[column].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax1 = plt.ylim(bottom=-1, top=1)
    else:
        ax1 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.title('Metric {}'.format(column))

    plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right',
               borderaxespad=0, fancybox=True, framealpha=0.5)

    ax2 = plt.subplot(312)
    ax2 = df[column+'-avg-healthy'].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='')
    ax2 = df[column+'-rolling-non-overlapping-avg'].plot(
        color='C6', alpha=1, linewidth=1, label='rolling average')

    plt.xlabel('')  # override

    upperBound = df[column+'-rolling-non-overlapping-avg'].max()
    lowerBound = df[column+'-rolling-non-overlapping-avg'].min()
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax2 = plt.ylim(bottom=-1, top=1)
    else:
        ax2 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))
    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=False)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)

    ax3 = plt.subplot(313)
    ax3 = plt.axvline(df.index[120], color='C2',
                      linewidth=0)  # triggering malware
    ax3 = df[column+'-avg-healthy'].plot(
        color='C4', alpha=1, linestyle='-', linewidth=1.75, label='average')

    ax3.xaxis.set_major_locator(mdates.MinuteLocator())
    ax3.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))

    # adjust y axis
    if float(lowerBound) == 0 and float(upperBound) == 0:
        ax3 = plt.ylim(bottom=-1, top=1)
    else:
        ax3 = plt.ylim(bottom=math.floor(lowerBound-0.5*(upperBound-lowerBound)),
                       top=math.ceil(upperBound+0.5*(upperBound-lowerBound)))

    plt.tick_params(axis='x', which='both', bottom=True,
                    top=False, labelbottom=True)
    plt.locator_params(axis='y', nbins=6)
    plt.locator_params(axis='x', nbins=6)
    #plt.legend(bbox_to_anchor=(0.98, 0.98), loc='upper right', borderaxespad=0, fancybox=True, framealpha=0.5)
    plt.xlabel('')  # override
    #plt.legend(loc='upper left')

    fig.subplots_adjust(hspace=0.2, left=0.14, right=0.90)
    plt.setp([a.get_xticklabels() for a in fig.axes[:-1]], visible=True)

    lines_labels = [ax.get_legend_handles_labels() for ax in fig.axes]
    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
    plt.legend(lines[1:], labels[1:], bbox_to_anchor=(
        0.0, -0.5), loc='upper left', borderaxespad=0, fancybox=True, framealpha=0.5)

    plt.savefig('plots/timeseries/{}/{}-{}-timeseries.pdf'.format(name, name[3:], column,),
                facecolor='white', transparent=False, bbox_inches="tight")
    index += 1

    plt.cla()
plt.close('all')
